# 🚀 Week 10 — Group Session: Advanced SQL and Integration

```
╔══════════════════════════════════════════════════════════════════╗
║  🤝 COLLABORATIVE SQL DEVELOPMENT                                ║
║                                                                  ║
║  "Together, we build better databases."                         ║
║                                                                  ║
║  This session covers:                                            ║
║  • Complex multi-table queries                                  ║
║  • Transaction management                                       ║
║  • Error handling patterns                                      ║
║  • Building data access APIs                                    ║
╚══════════════════════════════════════════════════════════════════╝
```

### 🎯 Session Format: Team-Based Development

| Role | Responsibility |
|------|----------------|
| ✍️ **Query Author** | Write SQL queries, design schemas |
| 🔌 **Integrator** | Connect Python and SQL, build functions |
| 🧪 **Tester** | Test edge cases, verify results |

> 💡 **Rotate roles** after each activity!

## 📋 Learning Objectives

By the end of this session, you will be able to:

| Objective | Skills |
|-----------|--------|
| 🔗 **Write Complex Queries** | Multi-table JOINs, aggregations |
| 🔒 **Handle Transactions** | BEGIN, COMMIT, ROLLBACK |
| ⚠️ **Implement Error Handling** | try/except with rollback |
| 🐍 **Build Data APIs** | Repository pattern, type safety |

### Prerequisites:
- ✅ Completed Session 1 (Advanced SQL concepts)
- ✅ Understanding of JOINs and aggregations
- ✅ Familiarity with Python functions

---

## 🔗 Activity 1: Complex JOINs with Aggregation

```
╔══════════════════════════════════════════════════════════════════╗
║  Understanding LEFT JOIN vs INNER JOIN                          ║
║                                                                  ║
║  INNER JOIN:    Only matching rows (users WITH orders)          ║
║  LEFT JOIN:     All left rows + matches (ALL users)             ║
║                                                                  ║
║  users (left)     orders (right)                                ║
║  ┌────┐           ┌────┐                                        ║
║  │ 1  │═══════════│ 1  │  INNER: Only 1, 2                     ║
║  │ 2  │═══════════│ 2  │  LEFT: 1, 2, 3 (3 with NULLs)         ║
║  │ 3  │           │    │                                        ║
║  └────┘           └────┘                                        ║
╚══════════════════════════════════════════════════════════════════╝
```

### COALESCE for NULL Handling:

```sql
-- COALESCE returns the first non-NULL value
SELECT 
    name,
    COALESCE(SUM(total), 0) as total  -- 0 if NULL
FROM users LEFT JOIN orders ON ...
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🚫 BAD PRACTICE: Missing NULL Handling                          ║
# ╚══════════════════════════════════════════════════════════════════╝
# What happens when we don't handle NULLs properly

import sqlite3

conn = sqlite3.connect(':memory:')
conn.row_factory = sqlite3.Row
cur = conn.cursor()

# Create test data
cur.executescript('''
    CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT);
    CREATE TABLE orders (id INTEGER PRIMARY KEY, user_id INTEGER, total REAL);
    INSERT INTO users (name) VALUES ('Ana'), ('Bob'), ('Cara'), ('Dan');
    INSERT INTO orders (user_id, total) VALUES 
        (1, 10.0), (1, 15.0),   -- Ana: 2 orders
        (2, 9.0),               -- Bob: 1 order
        (3, 25.0), (3, 10.0);   -- Cara: 2 orders
    -- Dan: NO orders!
''')
conn.commit()

# ❌ Bad: Using INNER JOIN - Dan disappears!
print("❌ BAD: INNER JOIN (missing users):")
cur.execute('''
    SELECT u.name, COUNT(o.id), SUM(o.total) 
    FROM users u 
    INNER JOIN orders o ON u.id = o.user_id 
    GROUP BY u.id
''')
for row in cur.fetchall():
    print(f"  {row[0]}: {row[1]} orders, ${row[2]:.2f}")
print("  ⚠️ Where's Dan?!")

# ❌ Bad: LEFT JOIN but no NULL handling
print("\n❌ BAD: LEFT JOIN without COALESCE:")
cur.execute('''
    SELECT u.name, COUNT(o.id), SUM(o.total) 
    FROM users u 
    LEFT JOIN orders o ON u.id = o.user_id 
    GROUP BY u.id
''')
for row in cur.fetchall():
    total = row[2] if row[2] else "None (NULL)"
    print(f"  {row[0]}: {row[1]} orders, total: {total}")
print("  ⚠️ NULL is not the same as 0!")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🌱 NOVICE: Proper LEFT JOIN with COALESCE                       ║
# ╚══════════════════════════════════════════════════════════════════╝
# Using LEFT JOIN and COALESCE to include all users

print("🌱 NOVICE: LEFT JOIN with COALESCE\n")

# ✅ Better: LEFT JOIN includes all users
cur.execute('''
    SELECT 
        u.name,
        COUNT(o.id) as order_count,
        COALESCE(SUM(o.total), 0) as total_spend
    FROM users u 
    LEFT JOIN orders o ON u.id = o.user_id 
    GROUP BY u.id
    ORDER BY total_spend DESC
''')

print("All users with their order statistics:")
print("-" * 50)
for row in cur.fetchall():
    status = "✅" if row['order_count'] > 0 else "📭"
    print(f"  {status} {row['name']}: {row['order_count']} orders, ${row['total_spend']:.2f}")

# Dan now shows up with 0 orders and $0.00!

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔧 INTERMEDIATE: Rich User Statistics                           ║
# ╚══════════════════════════════════════════════════════════════════╝
# Building comprehensive user analytics queries

print("🔧 INTERMEDIATE: Rich User Statistics\n")

def get_user_statistics():
    """Get comprehensive statistics for all users."""
    query = '''
        SELECT 
            u.id,
            u.name,
            COUNT(o.id) as order_count,
            COALESCE(SUM(o.total), 0) as total_spend,
            COALESCE(AVG(o.total), 0) as avg_order,
            COALESCE(MIN(o.total), 0) as min_order,
            COALESCE(MAX(o.total), 0) as max_order,
            CASE 
                WHEN COUNT(o.id) = 0 THEN 'Inactive'
                WHEN SUM(o.total) >= 30 THEN 'VIP'
                WHEN SUM(o.total) >= 15 THEN 'Active'
                ELSE 'New'
            END as customer_tier
        FROM users u 
        LEFT JOIN orders o ON u.id = o.user_id 
        GROUP BY u.id
        ORDER BY total_spend DESC
    '''
    cur.execute(query)
    return [dict(row) for row in cur.fetchall()]

# Display rich statistics
print("📊 Comprehensive User Analytics:")
print("-" * 70)
print(f"{'Name':<10} {'Orders':<8} {'Total':<10} {'Avg':<10} {'Tier':<10}")
print("-" * 70)

for stats in get_user_statistics():
    print(f"{stats['name']:<10} "
          f"{stats['order_count']:<8} "
          f"${stats['total_spend']:<9.2f} "
          f"${stats['avg_order']:<9.2f} "
          f"{stats['customer_tier']:<10}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  ✨ PYTHONIC: Type-Safe Statistics Service                       ║
# ╚══════════════════════════════════════════════════════════════════╝
# Production-ready statistics with dataclasses and type hints

from dataclasses import dataclass
from typing import List, Optional
from enum import Enum

class CustomerTier(Enum):
    """Customer tier based on spending."""
    INACTIVE = "Inactive"
    NEW = "New"
    ACTIVE = "Active"
    VIP = "VIP"

@dataclass
class UserStatistics:
    """Statistics for a single user."""
    id: int
    name: str
    order_count: int
    total_spend: float
    avg_order: float
    min_order: float
    max_order: float
    tier: CustomerTier
    
    @property
    def is_active(self) -> bool:
        return self.order_count > 0

class StatisticsService:
    """Service for user statistics."""
    
    def __init__(self, connection: sqlite3.Connection):
        self.conn = connection
        self.conn.row_factory = sqlite3.Row
    
    def _determine_tier(self, order_count: int, total_spend: float) -> CustomerTier:
        """Determine customer tier based on activity."""
        if order_count == 0:
            return CustomerTier.INACTIVE
        elif total_spend >= 30:
            return CustomerTier.VIP
        elif total_spend >= 15:
            return CustomerTier.ACTIVE
        else:
            return CustomerTier.NEW
    
    def get_all_user_statistics(self) -> List[UserStatistics]:
        """Get statistics for all users."""
        cursor = self.conn.execute('''
            SELECT 
                u.id,
                u.name,
                COUNT(o.id) as order_count,
                COALESCE(SUM(o.total), 0) as total_spend,
                COALESCE(AVG(o.total), 0) as avg_order,
                COALESCE(MIN(o.total), 0) as min_order,
                COALESCE(MAX(o.total), 0) as max_order
            FROM users u 
            LEFT JOIN orders o ON u.id = o.user_id 
            GROUP BY u.id
            ORDER BY total_spend DESC
        ''')
        
        results = []
        for row in cursor.fetchall():
            tier = self._determine_tier(row['order_count'], row['total_spend'])
            results.append(UserStatistics(
                id=row['id'],
                name=row['name'],
                order_count=row['order_count'],
                total_spend=row['total_spend'],
                avg_order=row['avg_order'],
                min_order=row['min_order'],
                max_order=row['max_order'],
                tier=tier
            ))
        return results
    
    def get_tier_summary(self) -> dict:
        """Get count of users per tier."""
        stats = self.get_all_user_statistics()
        return {
            tier.value: sum(1 for s in stats if s.tier == tier)
            for tier in CustomerTier
        }

# ═══════════════════════════════════════════════════════════════════
# ✨ Demonstration
# ═══════════════════════════════════════════════════════════════════

print("✨ PYTHONIC: Type-Safe Statistics Service\n")

service = StatisticsService(conn)

print("👥 User Statistics:")
print("-" * 60)
for stats in service.get_all_user_statistics():
    icon = {"VIP": "🌟", "Active": "✅", "New": "🆕", "Inactive": "💤"}[stats.tier.value]
    print(f"  {icon} {stats.name} [{stats.tier.value}]")
    print(f"     {stats.order_count} orders, ${stats.total_spend:.2f} total")

print("\n📊 Tier Summary:")
for tier, count in service.get_tier_summary().items():
    print(f"  {tier}: {count} users")

---

## 🎯 Group Activity 1: Zero-Count Rows Query

### Your Task:

Write a query that **explicitly shows users with zero orders** as having:
- `order_count = 0`
- `total_spend = 0.00`

### Requirements:
1. Include ALL users (use LEFT JOIN)
2. Handle NULLs with COALESCE
3. Show 0 for users without orders (not NULL)
4. Order by total_spend descending

### 💡 Hints:

```sql
SELECT 
    u.name,
    COUNT(o.id) as order_count,          -- COUNT handles NULL automatically
    COALESCE(SUM(o.total), 0) as total   -- COALESCE converts NULL to 0
FROM users u
LEFT JOIN orders o ON ...
GROUP BY ...
```

### Discussion:
- Why does COUNT(o.id) return 0 but SUM(o.total) returns NULL?
- What's the difference between COUNT(*) and COUNT(o.id)?

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  👩‍💻 YOUR WORKSPACE: Zero-Count Rows Query                        ║
# ╚══════════════════════════════════════════════════════════════════╝
# Write a query that shows ALL users with their order statistics

# ═══════════════════════════════════════════════════════════════════
# 📝 YOUR CODE HERE
# ═══════════════════════════════════════════════════════════════════

your_query = '''
    SELECT 
        u.name,
        -- Add COUNT for order_count
        -- Add COALESCE(SUM(...), 0) for total_spend
    FROM users u
    -- Add LEFT JOIN
    -- Add GROUP BY
    -- Add ORDER BY
'''

# Uncomment to test:
# cur.execute(your_query)
# for row in cur.fetchall():
#     print(f"{row['name']}: {row['order_count']} orders, ${row['total_spend']:.2f}")

# ═══════════════════════════════════════════════════════════════════
# 💡 SOLUTION
# ═══════════════════════════════════════════════════════════════════

solution_query = '''
    SELECT 
        u.name,
        COUNT(o.id) as order_count,
        COALESCE(SUM(o.total), 0) as total_spend
    FROM users u
    LEFT JOIN orders o ON u.id = o.user_id
    GROUP BY u.id
    ORDER BY total_spend DESC
'''

print("✅ Solution: All Users with Zero-Count Handling\n")
cur.execute(solution_query)
for row in cur.fetchall():
    status = "📦" if row['order_count'] > 0 else "📭"
    print(f"  {status} {row['name']}: {row['order_count']} orders, ${row['total_spend']:.2f}")

# ═══════════════════════════════════════════════════════════════════
# 📝 Discussion Answers:
# ═══════════════════════════════════════════════════════════════════

print("\n💡 Why COUNT vs SUM behaves differently:")
print("  • COUNT(o.id) counts non-NULL values → 0 when no matches")
print("  • SUM(o.total) returns NULL when all values are NULL")
print("  • COALESCE(SUM(o.total), 0) converts NULL to 0")
print("\n  COUNT(*) vs COUNT(column):")
print("  • COUNT(*) counts all rows (even with NULLs)")
print("  • COUNT(column) counts only non-NULL values")

---

## 🔒 Activity 2: Transactions and Error Handling

```
╔══════════════════════════════════════════════════════════════════╗
║  Transaction Error Handling Pattern                              ║
║                                                                  ║
║  try:                                                            ║
║      BEGIN                                                       ║
║      ... operations ...                                          ║
║      COMMIT  ✅                                                  ║
║  except:                                                         ║
║      ROLLBACK  ↩️                                                 ║
║                                                                  ║
║  Why it matters:                                                ║
║  • Atomicity: All or nothing                                    ║
║  • Consistency: Data stays valid                                ║
║  • Recovery: Undo failed operations                             ║
╚══════════════════════════════════════════════════════════════════╝
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🚫 BAD PRACTICE: No Transaction Safety                          ║
# ╚══════════════════════════════════════════════════════════════════╝
# What NOT to do with multi-step operations

def add_order_bad(name: str, total: float) -> bool:
    """Add an order - BAD way (no transaction safety)."""
    # ❌ Bad: No transaction - partial failure possible!
    
    # Step 1: Find or create user
    cur.execute('SELECT id FROM users WHERE name = ?', (name,))
    row = cur.fetchone()
    
    if not row:
        cur.execute('INSERT INTO users (name) VALUES (?)', (name,))
        user_id = cur.lastrowid
    else:
        user_id = row[0] if isinstance(row, tuple) else row['id']
    
    # 💥 What if the program crashes here?
    # User is created but order isn't!
    
    cur.execute('INSERT INTO orders (user_id, total) VALUES (?, ?)', (user_id, total))
    conn.commit()
    
    return True

print("❌ BAD: No transaction protection")
print("   If crash happens between INSERT statements,")
print("   data becomes inconsistent!")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🌱 NOVICE: Basic Transaction with try/except                    ║
# ╚══════════════════════════════════════════════════════════════════╝
# Using try/except with rollback for safety

def add_order_novice(connection, name: str, total: float) -> bool:
    """
    Add an order with basic transaction safety.
    
    Args:
        connection: Database connection
        name: User's name
        total: Order total
        
    Returns:
        True if successful, False otherwise
    """
    try:
        cursor = connection.cursor()
        cursor.execute('BEGIN')  # Start transaction
        
        # Find or create user
        cursor.execute('SELECT id FROM users WHERE name = ?', (name,))
        row = cursor.fetchone()
        
        if not row:
            cursor.execute('INSERT INTO users (name) VALUES (?)', (name,))
            user_id = cursor.lastrowid
        else:
            user_id = row[0] if isinstance(row, tuple) else row['id']
        
        # Create order
        cursor.execute(
            'INSERT INTO orders (user_id, total) VALUES (?, ?)', 
            (user_id, total)
        )
        
        connection.commit()  # Save all changes
        return True
        
    except Exception as e:
        connection.rollback()  # Undo all changes
        print(f"❌ Transaction failed: {e}")
        return False

# ═══════════════════════════════════════════════════════════════════
# 🌱 Demonstration
# ═══════════════════════════════════════════════════════════════════

print("🌱 NOVICE: Basic Transaction Safety\n")

# Test successful transaction
success = add_order_novice(conn, 'Eve', 50.00)
print(f"Adding order for Eve: {'✅ Success' if success else '❌ Failed'}")

# Verify
cur.execute('SELECT name FROM users WHERE name = ?', ('Eve',))
result = cur.fetchone()
print(f"Eve exists: {'Yes' if result else 'No'}")

cur.execute('SELECT COUNT(*) FROM orders WHERE total = 50.00')
print(f"Order exists: {cur.fetchone()[0] > 0}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  🔧 INTERMEDIATE: Context Manager for Transactions               ║
# ╚══════════════════════════════════════════════════════════════════╝
# Using context managers for cleaner transaction handling

from contextlib import contextmanager
from typing import Optional, Tuple

@contextmanager
def transaction(connection):
    """
    Context manager for database transactions.
    Automatically commits on success, rolls back on failure.
    """
    try:
        yield connection
        connection.commit()
    except Exception:
        connection.rollback()
        raise

def add_order_intermediate(
    connection, 
    name: str, 
    total: float
) -> Tuple[bool, Optional[int]]:
    """
    Add an order using context manager.
    
    Returns:
        Tuple of (success, order_id or None)
    """
    try:
        with transaction(connection):
            cursor = connection.cursor()
            
            # Find or create user
            cursor.execute('SELECT id FROM users WHERE name = ?', (name,))
            row = cursor.fetchone()
            
            if not row:
                cursor.execute('INSERT INTO users (name) VALUES (?)', (name,))
                user_id = cursor.lastrowid
            else:
                user_id = row[0] if isinstance(row, tuple) else row['id']
            
            # Create order
            cursor.execute(
                'INSERT INTO orders (user_id, total) VALUES (?, ?)',
                (user_id, total)
            )
            order_id = cursor.lastrowid
            
            return True, order_id
            
    except Exception as e:
        return False, None

# ═══════════════════════════════════════════════════════════════════
# 🔧 Demonstration
# ═══════════════════════════════════════════════════════════════════

print("🔧 INTERMEDIATE: Context Manager Transactions\n")

success, order_id = add_order_intermediate(conn, 'Frank', 75.00)
print(f"Adding order for Frank: {'✅ Success' if success else '❌ Failed'}")
if order_id:
    print(f"Order ID: {order_id}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  ✨ PYTHONIC: Enterprise Order Service                           ║
# ╚══════════════════════════════════════════════════════════════════╝
# Production-ready order service with full error handling

from dataclasses import dataclass
from typing import List, Optional, Union
from enum import Enum, auto

class OrderResult(Enum):
    """Result of an order operation."""
    SUCCESS = auto()
    USER_NOT_FOUND = auto()
    INVALID_AMOUNT = auto()
    DATABASE_ERROR = auto()

@dataclass
class OrderResponse:
    """Response from order operations."""
    result: OrderResult
    order_id: Optional[int] = None
    user_id: Optional[int] = None
    message: str = ""
    
    @property
    def success(self) -> bool:
        return self.result == OrderResult.SUCCESS

class OrderService:
    """Enterprise order service with transaction safety."""
    
    def __init__(self, connection: sqlite3.Connection):
        self.conn = connection
        self.conn.row_factory = sqlite3.Row
    
    @contextmanager
    def _transaction(self):
        """Transaction context manager with automatic cleanup."""
        try:
            yield self.conn
            self.conn.commit()
        except Exception:
            self.conn.rollback()
            raise
    
    def _get_or_create_user(self, cursor, name: str) -> int:
        """Get user ID, creating user if needed."""
        cursor.execute('SELECT id FROM users WHERE name = ?', (name,))
        row = cursor.fetchone()
        
        if row:
            return row['id']
        
        cursor.execute('INSERT INTO users (name) VALUES (?)', (name,))
        return cursor.lastrowid
    
    def add_order(self, name: str, total: float) -> OrderResponse:
        """
        Add an order for a user.
        
        Args:
            name: User's name
            total: Order total amount
            
        Returns:
            OrderResponse with result details
        """
        # Validate input
        if total <= 0:
            return OrderResponse(
                result=OrderResult.INVALID_AMOUNT,
                message=f"Invalid amount: ${total:.2f}"
            )
        
        try:
            with self._transaction():
                cursor = self.conn.cursor()
                
                # Get or create user
                user_id = self._get_or_create_user(cursor, name)
                
                # Create order
                cursor.execute(
                    'INSERT INTO orders (user_id, total) VALUES (?, ?)',
                    (user_id, total)
                )
                order_id = cursor.lastrowid
                
                return OrderResponse(
                    result=OrderResult.SUCCESS,
                    order_id=order_id,
                    user_id=user_id,
                    message=f"Order #{order_id} created successfully"
                )
                
        except sqlite3.IntegrityError as e:
            return OrderResponse(
                result=OrderResult.DATABASE_ERROR,
                message=f"Integrity error: {e}"
            )
        except Exception as e:
            return OrderResponse(
                result=OrderResult.DATABASE_ERROR,
                message=f"Database error: {e}"
            )
    
    def add_multiple_orders(
        self, 
        orders: List[tuple]
    ) -> List[OrderResponse]:
        """
        Add multiple orders in a single transaction.
        All succeed or all fail.
        """
        responses = []
        
        try:
            with self._transaction():
                cursor = self.conn.cursor()
                
                for name, total in orders:
                    if total <= 0:
                        raise ValueError(f"Invalid amount for {name}: ${total:.2f}")
                    
                    user_id = self._get_or_create_user(cursor, name)
                    cursor.execute(
                        'INSERT INTO orders (user_id, total) VALUES (?, ?)',
                        (user_id, total)
                    )
                    
                    responses.append(OrderResponse(
                        result=OrderResult.SUCCESS,
                        order_id=cursor.lastrowid,
                        user_id=user_id,
                        message="Success"
                    ))
                
                return responses
                
        except Exception as e:
            # All failed - return error for all
            return [
                OrderResponse(
                    result=OrderResult.DATABASE_ERROR,
                    message=f"Batch failed: {e}"
                )
                for _ in orders
            ]
    
    def get_user_summary(self, name: str) -> Optional[dict]:
        """Get summary for a user."""
        cursor = self.conn.execute('''
            SELECT 
                u.name,
                COUNT(o.id) as order_count,
                COALESCE(SUM(o.total), 0) as total_spend
            FROM users u
            LEFT JOIN orders o ON u.id = o.user_id
            WHERE u.name = ?
            GROUP BY u.id
        ''', (name,))
        
        row = cursor.fetchone()
        return dict(row) if row else None

# ═══════════════════════════════════════════════════════════════════
# ✨ Demonstration
# ═══════════════════════════════════════════════════════════════════

print("✨ PYTHONIC: Enterprise Order Service\n")

service = OrderService(conn)

# Single order
print("📦 Adding single order:")
response = service.add_order('Grace', 125.00)
print(f"  {response.message}")
print(f"  Success: {response.success}, Order ID: {response.order_id}")

# Invalid order (negative amount)
print("\n📦 Attempting invalid order:")
response = service.add_order('Grace', -50.00)
print(f"  Result: {response.result.name}")
print(f"  Message: {response.message}")

# Batch orders
print("\n📦 Adding batch orders:")
responses = service.add_multiple_orders([
    ('Henry', 30.00),
    ('Ivy', 45.00),
    ('Jack', 20.00)
])
for r in responses:
    print(f"  Order #{r.order_id}: {r.message}")

# Summary
print("\n📊 User Summaries:")
for name in ['Grace', 'Henry', 'Ivy']:
    summary = service.get_user_summary(name)
    if summary:
        print(f"  {summary['name']}: {summary['order_count']} orders, "
              f"${summary['total_spend']:.2f}")

---

## 🎯 Group Activity 2: Batch Insert with Reporting

### Your Task:

Create a function that:
1. Accepts a list of (name, total) tuples
2. Inserts ALL in a single transaction
3. Reports total orders per user AFTER all inserts

### Requirements:
- Use a single transaction for atomicity
- If ANY insert fails, roll back ALL
- Return a summary of each user's order count

### 💡 Hints:

```python
def batch_insert_with_report(orders: List[tuple]) -> dict:
    # 1. Start transaction
    # 2. Loop through orders, inserting each
    # 3. Commit all at once
    # 4. Query for summary
    # 5. Return results
    pass
```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║  👩‍💻 YOUR WORKSPACE: Batch Insert with Reporting                  ║
# ╚══════════════════════════════════════════════════════════════════╝
# Create a function that batch inserts and reports

from typing import List, Dict

# ═══════════════════════════════════════════════════════════════════
# 📝 YOUR CODE HERE
# ═══════════════════════════════════════════════════════════════════

def batch_insert_with_report_attempt(
    orders: List[tuple]
) -> Dict[str, int]:
    """
    Insert multiple orders and report totals per user.
    
    Args:
        orders: List of (name, total) tuples
        
    Returns:
        Dict mapping user names to their total order count
    """
    # TODO: Implement this function
    # 1. Start transaction with cur.execute('BEGIN')
    # 2. For each order, find/create user and insert order
    # 3. Commit with conn.commit()
    # 4. Query for order counts per user
    # 5. Return as dictionary
    pass

# ═══════════════════════════════════════════════════════════════════
# 💡 SOLUTION
# ═══════════════════════════════════════════════════════════════════

def batch_insert_with_report(orders: List[tuple]) -> Dict[str, int]:
    """
    Insert multiple orders in a single transaction and report totals.
    
    Args:
        orders: List of (name, total) tuples
        
    Returns:
        Dict mapping user names to their total order count
    """
    user_ids = {}  # Track user IDs for the report
    
    try:
        cur.execute('BEGIN')
        
        for name, total in orders:
            # Find or create user
            cur.execute('SELECT id FROM users WHERE name = ?', (name,))
            row = cur.fetchone()
            
            if row:
                user_id = row['id']
            else:
                cur.execute('INSERT INTO users (name) VALUES (?)', (name,))
                user_id = cur.lastrowid
            
            user_ids[name] = user_id
            
            # Insert order
            cur.execute(
                'INSERT INTO orders (user_id, total) VALUES (?, ?)',
                (user_id, total)
            )
        
        conn.commit()
        
    except Exception as e:
        conn.rollback()
        print(f"❌ Batch failed: {e}")
        return {}
    
    # Get order counts for the users we just processed
    results = {}
    for name, user_id in user_ids.items():
        cur.execute(
            'SELECT COUNT(*) FROM orders WHERE user_id = ?',
            (user_id,)
        )
        results[name] = cur.fetchone()[0]
    
    return results

# ═══════════════════════════════════════════════════════════════════
# Test the solution
# ═══════════════════════════════════════════════════════════════════

print("✅ Testing batch_insert_with_report:\n")

# Test data
test_orders = [
    ('Karen', 15.00),
    ('Karen', 25.00),  # Same user, multiple orders
    ('Leo', 50.00),
    ('Mia', 30.00),
    ('Mia', 20.00),
    ('Mia', 10.00),   # Same user, 3 orders
]

result = batch_insert_with_report(test_orders)

print("Order counts after batch insert:")
for name, count in sorted(result.items()):
    print(f"  {name}: {count} orders")

---

## 🤝 Collaboration Guide

### Team Roles:

| Role | Responsibilities |
|------|------------------|
| ✍️ **Query Author** | Write SQL queries, design JOINs |
| 🔌 **Integrator** | Connect Python and SQL, handle transactions |
| 🧪 **Tester** | Test edge cases, validate results |

### Rotation Schedule:
- **Activity 1:** Practice JOINs and NULL handling
- **Activity 2:** Build transactional operations

### Discussion Topics:

1. **JOINs:**
   - When to use LEFT vs INNER JOIN?
   - How do NULLs affect COUNT vs SUM?
   - Performance implications of different JOINs

2. **Transactions:**
   - What operations need transactions?
   - How to handle partial failures?
   - Testing rollback behavior

3. **Python Integration:**
   - When to use raw SQL vs ORM?
   - How to structure data access code?
   - Error handling best practices

---

## 📚 Session Recap

### What We Learned Today:

| Topic | Key Concept | Best Practice |
|-------|-------------|---------------|
| **LEFT JOIN** | Include all rows from left | Use for complete reports |
| **COALESCE** | Handle NULL values | `COALESCE(SUM(x), 0)` |
| **Transactions** | Atomic operations | try/except with rollback |
| **Context Managers** | Clean resource handling | `with transaction():` |
| **Service Pattern** | Business logic layer | Separate from data access |

### The Evolution of Our Code:

| Level | Pattern |
|-------|---------|
| 🚫 **Bad** | No NULL handling, no transactions |
| 🌱 **Novice** | COALESCE, basic try/except |
| 🔧 **Intermediate** | Context managers, functions |
| ✨ **Pythonic** | Services, dataclasses, type hints |

### SQL Patterns Reference:

```sql
-- Include all users (even without orders)
SELECT u.name, COALESCE(SUM(o.total), 0)
FROM users u LEFT JOIN orders o ON u.id = o.user_id
GROUP BY u.id;

-- Count vs Sum behavior
COUNT(column)  -- Returns 0 for no matches
SUM(column)    -- Returns NULL for no matches
COALESCE(SUM(column), 0)  -- Returns 0 for no matches
```

### 🎯 Key Takeaways:

1. **LEFT JOIN** for complete data sets
2. **COALESCE** for NULL safety
3. **Transactions** for multi-step operations
4. **Context managers** for clean code
5. **Service layer** for business logic

### 🔜 Next Steps:
- Advanced visualization techniques
- Dashboard building
- Data pipeline design
- Real-world project applications